# 수집해야 하는 리소스 가격들
* EC2
-  NAT Gateway
-  ELB
-  EBS
* EKS
* RDS/Aurora
* ElastiCache
* MSK
* EFS
* S3
* VPC

# 0. 관련 라이브러리 다운로드 및 모듈 불러오기

In [7]:
!pip install pandas requests

In [9]:

import pandas as pd
import json
import requests
import datetime
from pprint import pprint

In [15]:
pd.set_option("display.max_colwidth", 100)

# 1. AWS 가격정보 수집

In [4]:
def download_resource_price_json(resource: str, region: str):
    request_template = "https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/{resource}/current/{region}/index.json"
    url = request_template.format(resource=resource, region=region)
    response = requests.get(url)
    today = datetime.date.today().isoformat()
    with open(f"{resource}_{today}.json", "w") as f:
        f.write(response.text)

def download_prices(resources: list[str], region: str):
    for resource in resources:
        download_resource_price_json(resource, region)

In [94]:
download_resource_price_json("AmazonS3", "ap-northeast-2")
# "TimedStorage",

In [6]:
# 수집대상 리소스
resources = ["AmazonEC2", "AmazonRDS", "AmazonVPC", "AmazonMSK", "AmazonElastiCache"]
download_prices(resources, "ap-northeast-2")

# 2. 가격정보 전처리

## 2-1. EC2 가격정보 전처리 (EC2, NAT Gateway, ELB, EBS)

In [101]:
with open("AmazonEC2_2024-10-03.json", "r") as f:
    raw_ec2_price_data = json.load(f)

In [112]:
def flatten_price(data: dict, product_family: str):
    result = []
    
    price_data = data["terms"]["OnDemand"]
    
    for sku, value in data["products"].items():
        new_row = value["attributes"]
        if value.get("productFamily") != product_family:
            continue
        
        price_data_row = next(iter(price_data[sku].values()))
        price_data_row = next(iter(price_data_row["priceDimensions"].values()))
        new_row["description"] = price_data_row["description"]
        new_row["pricePerUnit"] = price_data_row["pricePerUnit"]["USD"]
        new_row["unit"] = price_data_row["unit"]
        result.append(new_row)

    return pd.DataFrame(result)

In [113]:
def filter_df(df: pd.DataFrame, filters: dict):
    for column, value in filters.items():
        if column not in df.columns:
            raise KeyError(f"데이터프레임에 '{column}' 컬럼이 존재하지 않습니다.")
        if isinstance(value, list):
            df = df[df[column].isin(value)]
        else:
            df = df[df[column] == value]
    return df

In [118]:
ec2_df = flatten_price(raw_ec2_price_data, "Compute Instance")

ec2_filters = {
    "operatingSystem": ["Linux", "Windows"],
    "tenancy": "Shared",
    "currentGeneration": "Yes",
    "preInstalledSw": "NA",
    "storage": "EBS only",
    "processorArchitecture": "64-bit",
    "capacitystatus": "Used",
    "marketoption": "OnDemand",
}
ec2_columns = [
    "instanceType",
    "instanceFamily",
    "vcpu",
    "memory",
    "gpuMemory",
    "operatingSystem",
    "physicalProcessor",
    "pricePerUnit",
    "unit",
]

ec2_df = filter_df(ec2_df, ec2_filters)
# 필터링된 컬럼만 선택
ec2_df = ec2_df[ec2_columns]
# 중복된 행 제거
ec2_df.drop_duplicates()

,instanceType,instanceFamily,vcpu,memory,gpuMemory,operatingSystem,physicalProcessor,pricePerUnit,unit
29,c7i.12xlarge,Compute optimized,48,96 GiB,NA,Linux,Intel Xeon Scalable (Sapphire Rapids),2.4192000000,Hrs
70,r5.4xlarge,Memory optimized,16,128 GiB,NA,Windows,Intel Xeon Platinum 8175,1.2160000000,Hrs
219,g3s.xlarge,GPU instance,4,30.5 GiB,8 GB,Windows,Intel Xeon E5-2686 v4 (Broadwell),1.1180000000,Hrs
228,r6i.16xlarge,Memory optimized,64,512 GiB,NA,Linux,Intel Xeon 8375C (Ice Lake),4.8640000000,Hrs
249,m6i.xlarge,General purpose,4,16 GiB,NA,Linux,Intel Xeon 8375C (Ice Lake),0.2360000000,Hrs
...,...,...,...,...,...,...,...,...,...
39425,inf1.xlarge,Machine Learning ASIC Instances,4,8 GiB,NA,Linux,Intel Xeon Platinum 8275CL (Cascade Lake),0.2810000000,Hrs
39485,c6i.8xlarge,Compute optimized,32,64 GiB,NA,Linux,Intel Xeon 8375C (Ice Lake),1.5360000000,Hrs
39596,r6i.4xlarge,Memory optimized,16,128 GiB,NA,Windows,Intel Xeon 8375C (Ice Lake),1.9520000000,Hrs
39764,m7i.2xlarge,General purpose,8,32 GiB,NA,Linux,Intel Xeon Scalable (Sapphire Rapids),0.4956000000,Hrs


In [119]:
ec2_df.drop_duplicates()

,instanceType,instanceFamily,vcpu,memory,gpuMemory,operatingSystem,physicalProcessor,pricePerUnit,unit
29,c7i.12xlarge,Compute optimized,48,96 GiB,NA,Linux,Intel Xeon Scalable (Sapphire Rapids),2.4192000000,Hrs
70,r5.4xlarge,Memory optimized,16,128 GiB,NA,Windows,Intel Xeon Platinum 8175,1.2160000000,Hrs
219,g3s.xlarge,GPU instance,4,30.5 GiB,8 GB,Windows,Intel Xeon E5-2686 v4 (Broadwell),1.1180000000,Hrs
228,r6i.16xlarge,Memory optimized,64,512 GiB,NA,Linux,Intel Xeon 8375C (Ice Lake),4.8640000000,Hrs
249,m6i.xlarge,General purpose,4,16 GiB,NA,Linux,Intel Xeon 8375C (Ice Lake),0.2360000000,Hrs
...,...,...,...,...,...,...,...,...,...
39425,inf1.xlarge,Machine Learning ASIC Instances,4,8 GiB,NA,Linux,Intel Xeon Platinum 8275CL (Cascade Lake),0.2810000000,Hrs
39485,c6i.8xlarge,Compute optimized,32,64 GiB,NA,Linux,Intel Xeon 8375C (Ice Lake),1.5360000000,Hrs
39596,r6i.4xlarge,Memory optimized,16,128 GiB,NA,Windows,Intel Xeon 8375C (Ice Lake),1.9520000000,Hrs
39764,m7i.2xlarge,General purpose,8,32 GiB,NA,Linux,Intel Xeon Scalable (Sapphire Rapids),0.4956000000,Hrs


In [121]:
# EBS 가격정보 dataframe으로 변환

ebs_df = flatten_price(raw_ec2_price_data, "Storage")
ebs_filters = {
    "volumeApiName": ["gp2", "gp3"],
}
ebs_df = filter_df(ebs_df, ebs_filters)

In [122]:
ebs_df

,servicecode,location,locationType,storageMedia,volumeType,maxVolumeSize,maxIopsvolume,maxThroughputvolume,usagetype,operation,regionCode,servicename,volumeApiName,description,pricePerUnit,unit,maxIopsBurstPerformance
0,AmazonEC2,Asia Pacific (Seoul),AWS Region,SSD-backed,General Purpose,16 TiB,16000,1000 MiB/s,APN2-EBS:VolumeUsage.gp3,,ap-northeast-2,Amazon Elastic Compute Cloud,gp3,$0.0912 per GB-month of General Purpose (gp3) provisioned storage - Asia Pacific (Seoul),0.0912000000,GB-Mo,NaN
5,AmazonEC2,Asia Pacific (Seoul),AWS Region,SSD-backed,General Purpose,16 TiB,16000,250 MiB/s,APN2-EBS:VolumeUsage.gp2,,ap-northeast-2,Amazon Elastic Compute Cloud,gp2,$0.114 per GB-month of General Purpose SSD (gp2) provisioned storage - Asia Pacific (Seoul),0.1140000000,GB-Mo,3000 for volumes <= 1 TiB


In [39]:
# Load Balancer 가격정보 dataframe으로 변환
alb_df = flatten_price(raw_ec2_price_data, "Load Balancer-Application")
alb_filters = {"usagetype": "APN2-LoadBalancerUsage"}
alb_df = filter_df(alb_df, alb_filters)
alb_df

,servicecode,location,locationType,group,groupDescription,usagetype,operation,regionCode,servicename,description,pricePerUnit,unit
0,AmazonEC2,Asia Pacific (Seoul),AWS Region,ELB:Balancer,LoadBalancer hourly usage by Application Load Balancer,APN2-LoadBalancerUsage,LoadBalancing:Application,ap-northeast-2,Amazon Elastic Compute Cloud,$7.0928 per Unused Reservation RHEL with SQL Standard r6id.8xlarge Instance Hour,7.0928000000,Hrs
1,AmazonEC2,Asia Pacific (Seoul),AWS Region,ELB:Balancer,Used Application load balancer capacity units-hr,APN2-LCUUsage,LoadBalancing:Application,ap-northeast-2,Amazon Elastic Compute Cloud,$7.0928 per Unused Reservation RHEL with SQL Standard r6id.8xlarge Instance Hour,7.0928000000,Hrs


In [40]:
nlb_df = flatten_price(raw_ec2_price_data, "Load Balancer-Network")
nlb_filters = {"usagetype": "APN2-LoadBalancerUsage"}
nlb_df = filter_df(nlb_df, nlb_filters)
nlb_df

,servicecode,location,locationType,group,groupDescription,usagetype,operation,regionCode,servicename,description,pricePerUnit,unit
0,AmazonEC2,Asia Pacific (Seoul),AWS Region,ELB:Balancer,LoadBalancer hourly usage by Network Load Balancer,APN2-LoadBalancerUsage,LoadBalancing:Network,ap-northeast-2,Amazon Elastic Compute Cloud,$7.0928 per Unused Reservation RHEL with SQL Standard r6id.8xlarge Instance Hour,7.0928000000,Hrs
1,AmazonEC2,Asia Pacific (Seoul),AWS Region,ELB:Balancer,Used Network load balancer capacity units-hr,APN2-LCUUsage,LoadBalancing:Network,ap-northeast-2,Amazon Elastic Compute Cloud,$7.0928 per Unused Reservation RHEL with SQL Standard r6id.8xlarge Instance Hour,7.0928000000,Hrs


In [41]:
nat_df = flatten_price(raw_ec2_price_data, "NAT Gateway")
nat_filter = {
    "unit": "Hrs",
    "usagetype": "APN2-NATGateway-Hours",
}
nat_df = filter_df(nat_df, nat_filter)

In [42]:
nat_df

,servicecode,location,locationType,group,groupDescription,usagetype,operation,regionCode,servicename,description,pricePerUnit,unit
0,AmazonEC2,Asia Pacific (Seoul),AWS Region,NGW:NatGateway,Charge for per GB data processed by NatGateways,APN2-NatGateway-Bytes,NatGateway,ap-northeast-2,Amazon Elastic Compute Cloud,$7.0928 per Unused Reservation RHEL with SQL Standard r6id.8xlarge Instance Hour,7.0928000000,Hrs
1,AmazonEC2,Asia Pacific (Seoul),AWS Region,NGW:NatGateway,Hourly charge for NAT Gateways,APN2-NatGateway-Hours,NatGateway,ap-northeast-2,Amazon Elastic Compute Cloud,$7.0928 per Unused Reservation RHEL with SQL Standard r6id.8xlarge Instance Hour,7.0928000000,Hrs


## 2-2. RDS/Aurora 가격정보 전처리

In [43]:
with open("AmazonRDS_2024-10-03.json", "r") as f:
    raw_rds_price_data = json.load(f)

In [49]:
rds_df = flatten_price(raw_rds_price_data, "Database Instance")

In [56]:
rds_df = rds_df.drop_duplicates(subset=["instanceType", "pricePerUnit"])
rds_columns = [
    "instanceType",
    "databaseEngine",
    "vcpu",
    "memory",
    "pricePerUnit",
    "unit",
]
rds_df = rds_df[rds_columns]

In [57]:
rds_df.head()

,instanceType,databaseEngine,vcpu,memory,pricePerUnit,unit
0,db.r5b.2xlarge.tpc1.mem2x,Oracle,8,128 GiB,13.9650000000,Hrs
2,db.m6gd.12xlarge,MySQL,48,192 GiB,13.9650000000,Hrs
3,db.r5d.24xlarge,MariaDB,96,768 GiB,13.9650000000,Hrs
4,db.r5.2xlarge,MariaDB,8,64 GiB,13.9650000000,Hrs
5,db.r5b.large,Oracle,2,16 GiB,13.9650000000,Hrs


In [51]:
len(rds_df)

2639

In [52]:
rds_filters = {
    "locationType": "AWS Region",
    "currentGeneration": "Yes",    
}

rds_df = filter_df(rds_df, rds_filters)


In [58]:
rds_df

,instanceType,databaseEngine,vcpu,memory,pricePerUnit,unit
0,db.r5b.2xlarge.tpc1.mem2x,Oracle,8,128 GiB,13.9650000000,Hrs
2,db.m6gd.12xlarge,MySQL,48,192 GiB,13.9650000000,Hrs
3,db.r5d.24xlarge,MariaDB,96,768 GiB,13.9650000000,Hrs
4,db.r5.2xlarge,MariaDB,8,64 GiB,13.9650000000,Hrs
5,db.r5b.large,Oracle,2,16 GiB,13.9650000000,Hrs
...,...,...,...,...,...,...
1742,db.r5.2xlarge.tpc1.mem2x,Oracle,8,128 GiB,13.9650000000,Hrs
1765,db.r5b.8xlarge.tpc2.mem3x,Oracle,32,768 GiB,13.9650000000,Hrs
1917,db.r6gd.2xlarge,Aurora PostgreSQL,8,64 GiB,13.9650000000,Hrs
2235,db.r6id.24xlarge,Aurora PostgreSQL,96,768 GiB,13.9650000000,Hrs


## 2-3. EKS 가격정보 전처리

In [88]:
with open("AmazonEKS_2024-10-03.json", "r") as f:
    raw_eks_price_data = json.load(f)

In [91]:
eks_df = flatten_price(raw_eks_price_data, "Compute")
eks_filters = {
    "locationType": "AWS Region",
    "usagetype": "APN2-AmazonEKS-Hours:perCluster",
    
}
eks_df = filter_df(eks_df, eks_filters)

In [92]:
eks_df

,servicecode,location,locationType,usagetype,operation,regionCode,servicename,storagetype,description,pricePerUnit,unit,tenancy,memorytype,tiertype,cputype
2,AmazonEKS,Asia Pacific (Seoul),AWS Region,APN2-AmazonEKS-Hours:perCluster,CreateOperation,ap-northeast-2,Amazon Elastic Container Service for Kubernetes,NaN,Amazon EKS local cluster usage on AWS Outposts,0.1000000000,hours,NaN,NaN,HAStandard,NaN


## 2-4. ElastiCache 가격정보 전처리

In [85]:
with open("AmazonElastiCache_2024-10-03.json", "r") as f:
    raw_redis_price_data = json.load(f)

In [78]:
redis_df = flatten_price(raw_redis_price_data, "Cache Instance")
redis_filters = {
    "locationType": "AWS Region",
    "currentGeneration": "Yes",
}
redis_df = filter_df(redis_df, redis_filters)

In [79]:
len(redis_df)

120

In [81]:
redis_df = redis_df.drop_duplicates(subset=["instanceType", "pricePerUnit"])
redis_columns = [
    "instanceType",
    "cacheEngine",
    "vcpu",
    "memory",
    "pricePerUnit",
    "unit",
]
redis_df = redis_df[redis_columns]

In [82]:
redis_df.head()

,instanceType,cacheEngine,vcpu,memory,pricePerUnit,unit
1,cache.m7g.4xlarge,Redis,16,52.26 GiB,0.0000000042,ElastiCacheProcessingUnit
2,cache.t2.small,Redis,1,1.55 GiB,0.0000000042,ElastiCacheProcessingUnit
6,cache.m5.2xlarge,Redis,8,26.04 GiB,0.0000000042,ElastiCacheProcessingUnit
7,cache.r4.2xlarge,Memcached,8,50.47 GiB,0.0000000042,ElastiCacheProcessingUnit
10,cache.r5.xlarge,Memcached,4,26.32 GiB,0.0000000042,ElastiCacheProcessingUnit
